Step 1: Import Libraries and Load Data

There are two beijing datasets. beijing.Rdata and beijing_clus.Rdata. beijing_clus.Rdata clusters the 315 regions into 26 regions and takes significantly less time to load. 

The path might need to be changed as the directory in the dropbox might not match what I used locally. 

In [2]:
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.stats import pearsonr

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt


pandas2ri.activate()
robjects.r['load']("./CrowdFlow/data/beijing_clus.RData")

'alldata'


Step 2: Process data

The original data is consisted of many tables. I seperated them into each individual dataframe so I could use them in python. 

Data_flow consists of information of the time index, region, inflow, outflow of the data. I added one extra column called trafiic_count which is the sum of inflow and outflow. 

Data_timemap consists of mappings of the 'time' column of the data_flow dataframe into actual time in the day, day in the week, hour in the day, hour in the week and the week number. 

Data_weather consists of weather information including temperature, pressure, humidity, windspeed, winddirection and weather (a 0~16 catagorical data for weather like sunny, rainy, snowy) for each time index. I was focusing on just the 'Weather' column of the data_weather dataframe. 

Data_flow_m is data_flow and data_weather merged together for easier processing. 

I also defined good weather as weather type 0 ~ 2 and bad weather is everything else. 

In [3]:
data = robjects.r['alldata']
data_flow = robjects.r['alldata'][0]
traffic_count = data_flow['outflow'] + data_flow['inflow']
data_flow['traffic_count'] = traffic_count
    
data_timemap = robjects.r['alldata'][1]
data_weather = robjects.r['alldata'][2]
testtime = robjects.r['alldata'][-1]
good_weather = data_weather.loc[data_weather['Weather'] <= 2]['time'].values.tolist()
# moderate_weather = data_weather[(data_weather['Weather'] > 2) & (data_weather['Weather'] <= 6)]['time'].values.tolist()
bad_weather = data_weather.loc[data_weather['Weather'] > 2]['time'].values.tolist()
weather_info = [-1 for _ in range (len(data_flow))]
# data_flow['Weather'] = data_weather['Weather']
data_flow['Weather_T'] = weather_info
# data_flow['Temperature'] = data_weather['Temperature']
data_flow.loc[data_flow['time'].isin(good_weather), 'Weather_T'] = 0
data_flow.loc[data_flow['time'].isin(bad_weather), 'Weather_T'] = 1
# data_flow.loc[data_flow['time'].isin(bad_weather), 'Weather'] = 2
# test_data = data_flow.loc[data_flow['time'] >= 5087].reset_index()
# train_data = data_flow.loc[data_flow['time'] < 5087].reset_index()
data_flow_m = data_flow.merge(data_weather, how='left', on='time')

Step 3. Run the Var Model

Since the Var model requires a complete time series, I just seperate the data into 26/315 region depending on which .Rdata I was using. Run Var model on each region, get the RMSE, repeat this for all regions and take the average. 

Since we are looking for RMSE for all weather types, good and bad weather, and the total, I seperated them into three different dataframes so its easier to keep track of. The total RMSE is tracked in df_RMSE_total, the good and bad weather RMSE is tracked in df_RMSE_goodandbad, and RMSE for all weather type for inflow, outflow and traffic_count is tracked under df_RMSE_inflow, df_RMSE_outflow, df_RMSE_TC respectively. 

While running the VAR model, I use the pearsonr test to choose the lag which has the smallest p-value. 

Then printing out the mean of each dataframe would get the RMSE we desire. 

In [4]:
index = list(range(1, 26))
# columns = ['MSE_IW_G', 'MSE_IW_B', 'MSE_IT_G', 'MSE_IT_B', 'MSE_OW_G', 'MSE_OW_B', 'MSE_OT_G', 'MSE_OT_B', 'MSE_TCW_G', 'MSE_TCW_B', 'MSE_TCT_G', 'MSE_TCT_B']
columns_goodandbad = ['MSE_Inflow_G', 'MSE_Inflow_B', 'MSE_Outflow_G', 'MSE_Outflow_B', 'MSE_TC_G', 'MSE_TC_B']
columns_total = ['MSE_Inflow', 'MSE_Outflow', 'MSE_TC']
columns_inflow = ['MSE_Inflow0', 'MSE_Inflow1', 'MSE_Inflow2', 'MSE_Inflow3', 'MSE_Inflow4', 'MSE_Inflow5', 'MSE_Inflow6', 'MSE_Inflow7', 'MSE_Inflow8', 'MSE_Inflow9', 'MSE_Inflow10', 'MSE_Inflow11', 'MSE_Inflow12', 'MSE_Inflow13', 'MSE_Inflow14', 'MSE_Inflow15', 'MSE_Inflow16']
columns_outflow = ['MSE_Outflow0', 'MSE_Outflow1', 'MSE_Outflow2', 'MSE_Outflow3', 'MSE_Outflow4', 'MSE_Outflow5', 'MSE_Outflow6', 'MSE_Outflow7', 'MSE_Outflow8', 'MSE_Outflow9', 'MSE_Outflow10', 'MSE_Outflow11', 'MSE_Outflow12', 'MSE_Outflow13', 'MSE_Outflow14', 'MSE_Outflow15', 'MSE_Outflow16']
columns_tc = ['MSE_TC0', 'MSE_TC1', 'MSE_TC2', 'MSE_TC3', 'MSE_TC4', 'MSE_TC5', 'MSE_TC6', 'MSE_TC7', 'MSE_TC8', 'MSE_TC9', 'MSE_TC10', 'MSE_TC11', 'MSE_TC12', 'MSE_TC13', 'MSE_TC14', 'MSE_TC15', 'MSE_TC16']
df_RMSE_total = pd.DataFrame(index = index, columns=columns_total)
df_RMSE_goodandbad = pd.DataFrame(index = index, columns=columns_goodandbad)
df_RMSE_inflow = pd.DataFrame(index=index, columns=columns_inflow)
df_RMSE_outflow = pd.DataFrame(index=index, columns=columns_outflow)
df_RMSE_TC = pd.DataFrame(index=index, columns=columns_tc)
df_RMSE_inflow = df_RMSE_inflow.fillna(0)
df_RMSE_outflow = df_RMSE_outflow.fillna(0)
df_RMSE_TC = df_RMSE_TC.fillna(0)

# print (df_RMSE.loc[1, columns])

for i in range (1, 27):

    if i == 24:
        i = i + 1

    data_flow_r1 = data_flow_m.loc[data_flow_m['region'] == i]
    time = data_flow_r1['time'].values
    test_data = data_flow_r1.loc[data_flow_r1['time'] >= 5087].reset_index()
    train_data = data_flow_r1.loc[data_flow_r1['time'] < 5087].reset_index()
    datetime = data_timemap[data_timemap['time'].isin(time)]['datetime']
    data_flow_r1 = data_flow_r1.loc[:, ["time", "outflow", "inflow", "traffic_count", "Weather", "Temperature", "Weather_T"]]
    data_flow_r1['datetime'] = datetime.values
    data_flow_r1.set_index('datetime', inplace = True, drop = True)
    data_flow_r1.dropna(inplace = True)
    data_flow_r1.index = pd.DatetimeIndex(data_flow_r1.index).to_period('min')

    # avgs = data_flow_r1.mean()
    # devs = data_flow_r1.std()
    # for i in range (1, 5):
    #     data_flow_r1.iloc[:, [i]] = (data_flow_r1.iloc[:, [i]] - avgs.iloc[i]) / devs.iloc[i]

    # t = data_flow_r1['time'].values

    # data_flow_r1 = data_flow_r1.diff()
    # data_flow_r1['time'] = t

    # data_flow_r1 = data_flow_r1.dropna()

    test_data_r1 = data_flow_r1.loc[data_flow_r1['time'] >= 5087].reset_index()
    train_data_r1 = data_flow_r1.loc[data_flow_r1['time'] < 5087].reset_index()

    RMSE_list_goodandbad = []
    RMSE_list_total = []
    RMSE_list_inflow = []
    RMSE_list_outflow = []
    RMSE_list_tc = []

    l = []
    for lag in range(1, 14):
        inflow_series = data_flow_r1['inflow'].iloc[lag:]
        lagged_weather_series = data_flow_r1['Weather'].iloc[:-lag]
        # print ('Lag: %s'%lag)
        # print (pearsonr(inflow_series, lagged_weather_series))
        l.append(pearsonr(inflow_series, lagged_weather_series)[1])
        # print ('----------')

    lag_IW = l.index(min(l)) + 1
    # print (lag_IW)

    df_IW = data_flow_r1[['inflow', 'Weather', 'Temperature']]
    model = VAR(df_IW)
    model_fit = model.fit(maxlags = lag_IW)
    # print (model_fit.summary())


    lag_order = model_fit.k_ar
    # print (lag_order)

    td_IW = train_data_r1[['inflow', 'Weather', 'Temperature']]
    fc = model_fit.forecast(td_IW.values[-lag_order:], len(test_data_r1))

    df_forecast = pd.DataFrame(fc, index=td_IW.index[-len(test_data_r1):], columns=td_IW.columns + '_forecast')
    # print (df_forecast)  


    test_data_r1['SSE_IW'] = [number ** 2 for number in (df_forecast['inflow_forecast'].values - test_data_r1['inflow'].values)]
    # print (test_data_r1)
    # print (test_data_r1.loc[test_data_r1['Weather_T'] == 0]['SSE_IW'].mean())
    # print (test_data_r1.loc[test_data_r1['Weather_T'] == 1]['SSE_IW'].mean())

    for j in range (0, 17):
        RMSE_list_inflow.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather'] == j]['SSE_IW'].mean()))

    RMSE_list_goodandbad.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 0]['SSE_IW'].mean()))
    RMSE_list_goodandbad.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 1]['SSE_IW'].mean()))

    RMSE_list_total.append(math.sqrt(test_data_r1['SSE_IW'].mean()))


    # l = []
    # for lag in range(1, 25):
    #     inflow_series = data_flow_r1['inflow'].iloc[lag:]
    #     lagged_temperature_series = data_flow_r1['Temperature'].iloc[:-lag]
    #     # print ('Lag: %s'%lag)
    #     # print (pearsonr(inflow_series, lagged_temperature_series))
    #     l.append(pearsonr(inflow_series, lagged_temperature_series)[1])
    #     # print ('----------')

    # lag_IT = l.index(min(l)) + 1
    # # print (lag_IT)

    # df_IT = data_flow_r1[['inflow', 'Temperature']]
    # model = VAR(df_IT)
    # model_fit = model.fit(lag_IT)
    # # print (model_fit.summary())


    # lag_order = model_fit.k_ar
    # # print (lag_order)

    # td_IT = train_data_r1[['inflow', 'Temperature']]
    # fc = model_fit.forecast(td_IT.values[-lag_order:], len(test_data_r1))

    # df_forecast = pd.DataFrame(fc, index=td_IT.index[-len(test_data_r1):], columns=td_IT.columns + '_forecast')
    # # print (df_forecast)  

    # test_data_r1['SSE_IT'] = [number ** 2 for number in (df_forecast['inflow_forecast'].values - test_data_r1['inflow'].values)]
    # # print (test_data_r1)
    # RMSE_list.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 0]['SSE_IT'].mean()))
    # RMSE_list.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 1]['SSE_IT'].mean()))

    l = []
    for lag in range(1, 14):
        outflow_series = data_flow_r1['outflow'].iloc[lag:]
        lagged_weather_series = data_flow_r1['Weather'].iloc[:-lag]
        # print ('Lag: %s'%lag)
        # print (pearsonr(outflow_series, lagged_weather_series))
        l.append(pearsonr(outflow_series, lagged_weather_series)[1])
        # print ('----------')

    lag_OW = l.index(min(l)) + 1
    # print (lag_OW)

    df_OW = data_flow_r1[['outflow', 'Weather', 'Temperature']]
    model = VAR(df_OW)
    model_fit = model.fit(lag_OW)
    # print (model_fit.summary())


    lag_order = model_fit.k_ar
    # print (lag_order)

    td_OW = train_data_r1[['outflow', 'Weather', 'Temperature']]
    fc = model_fit.forecast(td_OW.values[-lag_order:], len(test_data_r1))

    df_forecast = pd.DataFrame(fc, index=td_OW.index[-len(test_data_r1):], columns=td_OW.columns + '_forecast')
    # print (df_forecast)  

    test_data_r1['SSE_OW'] = [number ** 2 for number in (df_forecast['outflow_forecast'].values - test_data_r1['outflow'].values)]
    # print (test_data_r1)
    for j in range (0, 17):
        RMSE_list_outflow.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather'] == j]['SSE_OW'].mean()))    

    RMSE_list_goodandbad.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 0]['SSE_OW'].mean()))
    RMSE_list_goodandbad.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 1]['SSE_OW'].mean()))

    RMSE_list_total.append(math.sqrt(test_data_r1['SSE_OW'].mean()))

    # l = []
    # for lag in range(1, 14):
    #     outflow_series = data_flow_r1['outflow'].iloc[lag:]
    #     lagged_temperature_series = data_flow_r1['Temperature'].iloc[:-lag]
    #     # print ('Lag: %s'%lag)
    #     # print (pearsonr(outflow_series, lagged_weather_series))
    #     l.append(pearsonr(outflow_series, lagged_temperature_series)[1])
    #     # print ('----------')

    # lag_OT = l.index(min(l)) + 1
    # # print (lag_OT)

    # df_OT = data_flow_r1[['outflow', 'Temperature']]
    # model = VAR(df_OT)
    # model_fit = model.fit(lag_OT)
    # # print (model_fit.summary())


    # lag_order = model_fit.k_ar
    # # print (lag_order)

    # td_OT = train_data_r1[['outflow', 'Temperature']]
    # fc = model_fit.forecast(td_OT.values[-lag_order:], len(test_data_r1))

    # df_forecast = pd.DataFrame(fc, index=td_OT.index[-len(test_data_r1):], columns=td_OT.columns + '_forecast')
    # # print (df_forecast)  

    # test_data_r1['SSE_OT'] = [number ** 2 for number in (df_forecast['outflow_forecast'].values - test_data_r1['outflow'].values)]
    # # print (test_data_r1)
    # RMSE_list.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 0]['SSE_OT'].mean()))
    # RMSE_list.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 1]['SSE_OT'].mean()))

    l = []
    for lag in range(1, 14):
        tc_series = data_flow_r1['traffic_count'].iloc[lag:]
        lagged_weather_series = data_flow_r1['Weather'].iloc[:-lag]
        # print ('Lag: %s'%lag)
        # print (pearsonr(tc_series, lagged_weather_series))
        l.append(pearsonr(tc_series, lagged_weather_series)[1])
        # print ('----------')

    lag_TCW = l.index(min(l)) + 1
    # print (lag_TCW)

    df_TCW = data_flow_r1[['traffic_count', 'Weather', 'Temperature']]
    model = VAR(df_TCW)
    model_fit = model.fit(lag_TCW)
    # print (model_fit.summary())


    lag_order = model_fit.k_ar
    # print (lag_order)

    td_TCW = train_data_r1[['traffic_count', 'Weather', 'Temperature']]
    fc = model_fit.forecast(td_TCW.values[-lag_order:], len(test_data_r1))

    df_forecast = pd.DataFrame(fc, index=td_TCW.index[-len(test_data_r1):], columns=td_TCW.columns + '_forecast')
    # print (df_forecast)  

    test_data_r1['SSE_TCW'] = [number ** 2 for number in (df_forecast['traffic_count_forecast'].values - test_data_r1['traffic_count'].values)]
    # print (test_data_r1)
    for j in range (0, 17):
        RMSE_list_tc.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather'] == j]['SSE_TCW'].mean()))

    RMSE_list_goodandbad.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 0]['SSE_TCW'].mean()))
    RMSE_list_goodandbad.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 1]['SSE_TCW'].mean()))

    RMSE_list_total.append(math.sqrt(test_data_r1['SSE_TCW'].mean()))

    # l = []
    # for lag in range(1, 14):
    #     tc_series = data_flow_r1['traffic_count'].iloc[lag:]
    #     lagged_temperature_series = data_flow_r1['Temperature'].iloc[:-lag]
    #     # print ('Lag: %s'%lag)
    #     # print (pearsonr(tc_series, lagged_weather_series))
    #     l.append(pearsonr(tc_series, lagged_temperature_series)[1])
    #     # print ('----------')

    # lag_TCT = l.index(min(l)) + 1
    # # print (lag_TCT)

    # df_TCT = data_flow_r1[['traffic_count', 'Temperature']]
    # model = VAR(df_TCT)
    # model_fit = model.fit(lag_TCT)
    # # print (model_fit.summary())


    # lag_order = model_fit.k_ar
    # # print (lag_order)

    # td_TCT = train_data_r1[['traffic_count', 'Temperature']]
    # fc = model_fit.forecast(td_TCW.values[-lag_order:], len(test_data_r1))

    # df_forecast = pd.DataFrame(fc, index=td_TCT.index[-len(test_data_r1):], columns=td_TCT.columns + '_forecast')
    # # print (df_forecast)  

    # test_data_r1['SSE_TCT'] = [number ** 2 for number in (df_forecast['traffic_count_forecast'].values - test_data_r1['traffic_count'].values)]
    # # print (test_data_r1)
    # RMSE_list.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 0]['SSE_TCT'].mean()))
    # RMSE_list.append(math.sqrt(test_data_r1.loc[test_data_r1['Weather_T'] == 1]['SSE_TCT'].mean()))

    # print (RMSE_list)

    df_RMSE_inflow.loc[i] = RMSE_list_inflow
    df_RMSE_outflow.loc[i, columns_outflow] = RMSE_list_outflow
    df_RMSE_TC.loc[i, columns_tc] = RMSE_list_tc
    df_RMSE_total.loc[i, columns_total] = RMSE_list_total
    df_RMSE_goodandbad.loc[i, columns_goodandbad] = RMSE_list_goodandbad

# print (df_RMSE)
# print (df_RMSE_inflow)
# print (df_RMSE_outflow)
# print (df_RMSE_TC)


In [5]:
df_RMSE_inflow.mean()

MSE_Inflow0     48.220771
MSE_Inflow1     51.577266
MSE_Inflow2     32.251971
MSE_Inflow3      0.000000
MSE_Inflow4     42.909447
MSE_Inflow5      0.000000
MSE_Inflow6      0.000000
MSE_Inflow7      0.000000
MSE_Inflow8     39.980965
MSE_Inflow9      0.000000
MSE_Inflow10     0.000000
MSE_Inflow11     0.000000
MSE_Inflow12     0.000000
MSE_Inflow13     0.000000
MSE_Inflow14    37.406444
MSE_Inflow15     0.000000
MSE_Inflow16     0.000000
dtype: float64

In [6]:
df_RMSE_outflow.mean()

MSE_Outflow0     40.090480
MSE_Outflow1     49.144368
MSE_Outflow2     37.414375
MSE_Outflow3      0.000000
MSE_Outflow4     47.379418
MSE_Outflow5      0.000000
MSE_Outflow6      0.000000
MSE_Outflow7      0.000000
MSE_Outflow8     51.129009
MSE_Outflow9      0.000000
MSE_Outflow10     0.000000
MSE_Outflow11     0.000000
MSE_Outflow12     0.000000
MSE_Outflow13     0.000000
MSE_Outflow14    47.782362
MSE_Outflow15     0.000000
MSE_Outflow16     0.000000
dtype: float64

In [8]:
df_RMSE_TC.mean()

MSE_TC0     66.881935
MSE_TC1     82.812708
MSE_TC2     62.438834
MSE_TC3      0.000000
MSE_TC4     81.159455
MSE_TC5      0.000000
MSE_TC6      0.000000
MSE_TC7      0.000000
MSE_TC8     76.974950
MSE_TC9      0.000000
MSE_TC10     0.000000
MSE_TC11     0.000000
MSE_TC12     0.000000
MSE_TC13     0.000000
MSE_TC14    71.847257
MSE_TC15     0.000000
MSE_TC16     0.000000
dtype: float64

In [9]:
df_RMSE_total.mean()

MSE_Inflow     45.173873
MSE_Outflow    49.724875
MSE_TC         79.026322
dtype: float64

In [10]:
df_RMSE_goodandbad.mean()

MSE_Inflow_G     48.397009
MSE_Inflow_B     41.749728
MSE_Outflow_G    46.253003
MSE_Outflow_B    52.718181
MSE_TC_G         77.658423
MSE_TC_B         80.162662
dtype: float64